# Дата принятия

Термин «дата принятия» в рамках задачи используется не совсем юридически строго. Под датой принятия мы имеем в виду дату, которая используется, когда в ссылке на документ пишут: «во исполнение приказа No 123 от ...». Для законов это и будет дата принятия соответствующим законодательным органом, но для приказов или указов это будет дата подписания соответствующим уполномоченным лицом. У других типов НПА могут быть свои особенности.
Для некоторых типов документов вам могут встретиться сразу несколько дат. Например федеральные законы сначала принимаются Государственной Думой, потом одобряются Советом Федерации, а затем подписываются президентом. Извлекать нужно только дату принятия.

Поле date должно содержать дату принятия НПА, в указанном формате: dd.mm.yyyy.

# Для каждого типа документа свой извлекатор:

* федеральный закон - в конце документа, после "москва, кремль"

* указ - дата в конце 12 июня 2012 года

* постановление - дата в начале (от) 17.12.2015 г., 10 марта 2015 года 

* распоряжение - дата в начале от 27 ноября 2014 г.


In [1]:
import re
import json
import numpy as np

In [2]:
month2number = {
    'января': '01',
    'февраля': '02',
    'марта': '03',
    'апреля': '04',
    'мая': '05',
    'июня': '06',
    'июля': '07',
    'августа': '08',
    'сентября': '09',
    'октября': '10',
    'ноября': '11',
    'декабря': '12'
}

def process_match(match):
    day = match[0] if len(match[0]) == 2 else '0' + match[0]
    month = month2number[match[1]] if match[1] in month2number else match[1]
    year = match[2]
    return f"{day}.{month}.{year}"

def reverse_date(date):
    if len(date) == 10:
        return f"{date[6:]}.{date[3:5]}.{date[:2]}"
    return date

def find_max_date(variants, start=0):
    date = process_match(variants[0][start:]) if variants else ''
    for variant in variants:
        processed_variant = process_match(variant[start:])
        if reverse_date(processed_variant) > reverse_date(date):
            date = processed_variant
    return date

In [290]:
def doc_preprocess(doc):
    doc = '\n' + doc.lower() + '\n'
    doc = re.sub(r"[_о](\d?\d [а-я]+ \d\d\d\d)", r"\1", doc)
    doc = re.sub(r"_(\d?\d\.\d\d\.\d\d\d\d)", r"\1", doc)
    doc = re.sub(r"\n+", "\n", doc)
    return doc

In [291]:
date_reg_exprs = [
    re.compile(r'от (\d?\d) ([а-я]+) (\d\d\d\d)'),
    re.compile(r'\n(\d?\d) ([а-я]+) (\d\d\d\d)'),
    re.compile(r'(\d\d)\.(\d\d)\.(\d\d\d\d)'),
]

F_Z = re.compile(r'\n(\d?\d) ([а-я]+) (\d\d\d\d) года\n')
Z = re.compile(r'\n(от )?(\d?\d)[ \.]([а-я]+|\d\d)[ \.](\d\d\d\d)')
R = re.compile(r'\n(\d\d\.\d\d\.\d\d\d\d)[^\n]*\n№ [\w\-/а-я]+\n')
P = re.compile(r'постановление\n([^\n\d]*\n)?([^\n\d]*\n)?([^\n\d]*\n)?(от[_\- ]+?)?(\d?\d)[ \.]([а-я]+|\d\d)[ \.](\d\d\d\d)')
PRIKAZ = re.compile(r'приказ\n([^\n]*\n)?([^\n]*\n)?(от )?(\d?\d)[ \.]([а-я]+|\d\d)[ \.](\d\d\d\d)')
U = re.compile(r'указ\n([^\n]*\n)?([^\n]*\n)?([^\n]*\n)?(от )?(\d?\d)[ \.]([а-я]+|\d\d)[ \.](\d\d\d\d)')

In [355]:
def extract_date(doc, doc_type):
    doc = doc_preprocess(doc)
    if doc_type == "федеральный закон":
        match = F_Z.search(doc)
        if match:
            return process_match((match.group(1), match.group(2), match.group(3)))
        
    if doc_type == 'распоряжение':
        match = R.search(doc)
        if match:
            return match.group(1)
        
    if doc_type == 'закон':
        matches = Z.findall(doc)
        if matches:
            date = find_max_date(matches, start=1)
            return date
        
    if doc_type == 'постановление':
        doc = re.sub(r"[пнл]оста[нв]ов[пнл]е[нпл]и[ек]", r"постановление", doc)
        match = P.search(doc)
        if match:
            return process_match((match.group(5), match.group(6), match.group(7)))
        
    if doc_type == 'приказ':
        match = PRIKAZ.search(doc)
        if match:
            return process_match((match.group(4), match.group(5), match.group(6)))
              
    if doc_type == 'указ' and not doc.find('о внесении изменений') != -1:
        match = U.search(doc)
        if match:
            return process_match((match.group(5), match.group(6), match.group(7)))
        
    matches = []
    for reg_expr in date_reg_exprs:
        matches += reg_expr.findall(doc)
    if matches:
        if doc_type == 'постановление' and re.search(r"\\сессии\\", doc):
            matches = matches[:-1]
        date = find_max_date(matches)
        return date
    matches = re.findall(r'(\d?\d) ([а-я]+) (\d\d\d\d)', doc)
    if matches:
        date = find_max_date(matches)
        return date
    return ""

In [356]:
from eval_module import quality

def predict(test, types):
    results = []
    for doc, doc_type in zip(test, types):
        date = extract_date(doc, doc_type)
        prediction = {"type": "",
                      "date": date,
                      "number": "",
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])

true_types = np.array([x['type'] for x in labeled])
predicted = predict(train, true_types)
quality(predicted, labeled)

{'date_accuracy': 0.9944848333,
 'number_accuracy': 0.0,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [357]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
names = np.array(names)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}: {quality(predicted_labels[ids], test_labels[ids])['date_accuracy']}")

федеральный закон: 1.0
постановление: 0.9959333062
приказ: 0.987012987
распоряжение: 1.0
закон: 0.9849246231
указ: 0.9873417722


In [358]:
doc_type = 'указ'
ids = np.where(true_types == doc_type)

y_test = [(x['date'], x['type']) for x in test_labels[ids]]
y_pred = [x['date'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0] != pred:
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f.read())
        print("=====")

right value = ('26.11.2014', 'указ')
wrong value = 03.02.2006
Российская Федерация

Республика Карелия

УКАЗ
ГЛАВЫ РЕСПУБЛИКИ КАРЕЛИЯ

О внесении изменения в Указ Главы Республики Карелия
от 3 февраля 2006 года № 6

Внести в пункт 3 Положения о Межведомственной комиссии
по координации деятельности субъектов профилактики правонарушений
в Республике Карелия, утвержденного Указом Главы Республики Карелия
от 3 февраля 2006 года № 6 "О Межведомственной комиссии
по координации деятельности субъектов профилактики правонарушений
в Республике Карелия" (Собрание законодательства Республики Карелия,
2006, № 2, ст. 139; 2010, № 4, ст. 361; № 11, ст. 1403; 2011, № 10, ст. 1607),
изменение, заменив слова "заместитель Главы Республики Карелия -
Министр здравоохранения и социального развития Республики Карелия"
словами  "заместитель Главы Республики Карелия по социальным
вопросам".

 

Глава

Республики Карелх‹;х А.П. Худилайнен

а:
{ НО& рместе,
4ОЧ А ВМ т
* опК аС ЕИ е ост
Я‘ ъ ‚'‚:‘й‚"- ® {\_;; .д;